# Capse.jl chains
In this notebook we will show how to use the Capse.jl emulator in combination with Turing.jl, performing a ACTPolLite analysis. In particolar, we are gonna show:

- How to compute the Maximum A Posteriori (MAP), using the L-BFGS method provided by Optim.jl
- How to use Pathfinder.jl to obtain a quick posterior estimate and initialize chains
- How to use NUTS and MicroCanonical Hamiltonian MonteCarlo as sampling algorithms

Let us start initializing the Julia environment. This will download and install the precise version of the required packages.

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.resolve()

In [ ]:
using Statistics
using SimpleChains
using NPZ
using Turing
using Optim
using Pathfinder
using StatsPlots
using Capse
using BenchmarkTools
using LinearAlgebra
using JSON
using NPZ
using Transducers
using ACTPolLite
import MCMCChains: compute_duration
using MicroCanonicalHMC
using MCMCDiagnosticTools
using DataFrames
include("utils.jl")

Here we initialize the neural network we use for the emulator. This is done loading a dictionary, saved in a JSON file, that contains all the information required to instantiate the right NN.

In [ ]:
NN_dict = JSON.parsefile("nn_setup.json")

If you wanna see some details about the NN we created, just use

In [ ]:
Capse.get_emulator_description(NN_dict["emulator_description"])

After creating the neural network, we initialize our emulators.

In [ ]:
Capse.get_emulator_description(NN_dict["emulator_description"])

In [ ]:
weights_folder = "../data/weights/weights_cosmopowerspace_10000/"
ℓ = npzread(weights_folder*"l.npy")

weights_TT = npzread(weights_folder*"weights_TT_lcdm.npy")
trained_emu_TT = Capse.init_emulator(NN_dict, weights_TT, Capse.SimpleChainsEmulator)
CℓTT_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_TT, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓTT_lcdm.npy"));

In [ ]:
weights_EE = npzread(weights_folder*"weights_EE_lcdm.npy")
trained_emu_EE = Capse.init_emulator(NN_dict, weights_EE, Capse.SimpleChainsEmulator)
CℓEE_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_EE, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓEE_lcdm.npy"));

In [ ]:
weights_TE = npzread(weights_folder*"weights_TE_lcdm.npy")
trained_emu_TE = Capse.init_emulator(NN_dict, weights_TE, Capse.SimpleChainsEmulator)
CℓTE_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_TE, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓTE_lcdm.npy"));

In [ ]:
weights_PP = npzread(weights_folder*"weights_PP_lcdm.npy")
trained_emu_PP = Capse.init_emulator(NN_dict, weights_PP, Capse.SimpleChainsEmulator)
CℓPP_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_PP, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓPP_lcdm.npy"));

## ACTPolLite & Turing

Here we are going tyo create some functions to analyze ACTPol data.
The first function, given a list of arguments, retrieves the binned $C_\ell$'s 

In [ ]:
ls = 2:5000
fac=ls.*(ls.+1)./(2*π)

function call_emu_actpol(θ, Emu_TT, Emu_TE, Emu_EE, fac)
    return Capse.get_Cℓ(θ, Emu_TT)./fac, Capse.get_Cℓ(θ, Emu_TE)./fac, Capse.get_Cℓ(θ, Emu_EE)./fac
end

This other function is a closure, defining a more manageable version of the same function (in this way we don't have to pass some fixed arguments)

In [ ]:
theory_actpol(θ) = call_emu_actpol(θ, CℓTT_emu, CℓTE_emu, CℓEE_emu, fac)

In [ ]:
x = rand(6)
@benchmark theory_actpol($x)

Since the Covariance matrix we are gonna use does not depend on the model parameters, we can define the following quantities

$\Gamma = \mathrm{sqrt}(\Lambda)$

$i\Gamma=\mathrm{inv}(\Gamma)$

$D = i\Gamma \cdot d$

We can now sample a MvNormal with an easier covariance matrix

$D \sim \mathrm{MvNormal}(i\Gamma\cdot t(\theta), I)$

The advantage of this reparametrization is that we compute the inverse of a matrix just once and not at every step of the MCMC, without resoirting to any approximation: the two likelihood defined are mathematically equivalent.

In [ ]:
Γ = sqrt(ACTPolLite.cov_ACT)
iΓ = inv(Γ)
D = iΓ * ACTPolLite.data;

Although this is a bit different from the way we are used to code likelihoods in cosmology, it is easy to explain how to use a Probabilistic Programming Language (PPL) such as Turing.

When we use the "$\sim$" symbol, we are saying that the left-hand-side is sampled from the distribution on the right-hand-side

In [ ]:
@model function CMB_ACTPol(D, iΓ, WF)
    ln10As ~ Uniform(0.25, 0.35)
    ns     ~ Uniform(0.88, 1.06)
    h0     ~ Uniform(0.60, 0.80)
    ωb     ~ Uniform(0.1985, 0.25)
    ωc     ~ Uniform(0.08, 0.20)
    τ      ~ Normal(0.065, 0.015)
    yₚ     ~ Uniform(0.9, 1.1)

    
    θ = [10*ln10As, ns, 100*h0, ωb/10, ωc, τ]
    tt, te, ee = theory_actpol(θ)

    te .*= yₚ
    ee .*= yₚ^2
    
    X_model = iΓ * ACTPolLite.compone_window_Cℓ(tt, te, ee, WF)
    
    D ~ MvNormal(X_model, I)
end

CMB_ACTPol_model = CMB_ACTPol(D, iΓ, ACTPolLite.WF)

Let us perform the MAP computation. Turing has been interfaced with Optim, which provides some powerful minimization methods such as L-BFGS.

In [ ]:
bestfit_ACTPol = optimize(CMB_ACTPol_model, MAP(), Optim.Options(iterations=100000, allow_f_increases=true))
@benchmark optimize(CMB_ACTPol_model, MAP(), Optim.Options(iterations=100000, allow_f_increases=true))

The minimization takes less than half a second!

We are going to run 6 parallel chains, with 500 adaptation steps and 5'000 steps (note that Turing will not retrieve the burn-in steps)

In [ ]:
nsteps = 5000
nadapts = 500
nchains = 6

Before starting chains, let us use Pathfinder. This will deliver an approximation to the posterior

In [ ]:
result_multi = multipathfinder(CMB_ACTPol_model, 10000; nruns=8, executor=Transducers.PreferParallel())

In [ ]:
@time multipathfinder(CMB_ACTPol_model, 10000; nruns=8, executor=Transducers.PreferParallel())

In [ ]:
result_multi.draws_transformed

In [ ]:
chain_actpol_PF = combine_chains(result_multi.draws_transformed)
npzwrite("chains_ACT_PF.npy", chain_actpol_PF)

Pathfinder is incredibly fast: it performed the analysis in as few as 15 seconds. However, it is an approximate method. Although it might not always used to give a faithful approximation of the posteriori, it is very useful in starting the chains as close as possible to the typical set.

Here we are gonna use some Pathfinder draws to initialize our chains.

In [ ]:
init_params = collect.(eachrow(result_multi.draws_transformed.value[1:nchains, :, 1]));

Let us now start the NUTS chains.

In [ ]:
chains_actpol_nuts = sample(CMB_ACTPol_model, NUTS(nadapts, 0.65), MCMCThreads(), nsteps, nchains; init_params)

Let us plot our chains. As we can see, the samples looks almost uncorrelated, which is consistent with our estimate of the correlation length, which is between 2-3 for cosmological parameters.

In [ ]:
p = StatsPlots.plot(chains_actpol_nuts)
StatsPlots.savefig("traceplots_ACT.png")
p

In [ ]:
chain_act = combine_chains(chains_actpol_nuts)
npzwrite("chains_ACT_NUTS.npy", chain_act)

An interesting quantity we will compare with the MCHMC run is the ESS per second

In [ ]:
CPU_s_ACTPol_NUTS = compute_duration(chains_actpol_nuts)
ACTPol_NUTS_ESS = mean(MCMCDiagnosticTools.ess_rhat(chains_actpol_nuts)[[:ln10As, :ns, :h0, :ωb,:ωc, :τ, :yₚ],:ess])
ACTPol_NUTS_ESS_s = ACTPol_NUTS_ESS/CPU_s_ACTPol_NUTS

An ESS/s of 1.6 means that we can reach the (heuristic) threshold of 400 ESS in around 4 minutes and, taking advantage of the multiple processor, our analysis can be performed in around 1 minute.

## The MicroCanonical Hamiltonian MonteCarlo sampler
Here we will use the Julia implementation of the MCHMC sampler.
We will use 20'000 adaptation steps (that will be discarded) and 200'000 steps.

In [ ]:
d = 7
target = TuringTarget(CMB_ACTPol_model)
nadapts = 20_000
nsteps = 200_000
spl = MCHMC(nadapts, 0.001; init_eps=0.05, L=sqrt(d),# sigma=ones(d),
            adaptive=true)
@time actpol_mchmc = Sample(spl, target, nsteps;
                       progress=true,
                       dialog=true, file_name="chain_1",
                       initial_x=bestfit_ACTPol.values.array)

In [ ]:
n_parallel_mchmc = 8
chains = Vector{Any}(undef, n_parallel_mchmc)
vec_ess = zeros(n_parallel_mchmc)

nadapts = 20_000
nsteps = 200_000
start_mchmc = time()

@time for i in 1:n_parallel_mchmc
    chains[i] = Sample(MCHMC(nadapts, 0.001; init_eps=0.05, L=sqrt(d), adaptive=true), target, nsteps;
                       progress=true,
                       dialog=true, file_name="chain_1",
                       initial_x=bestfit_ACTPol.values.array)
    vec_ess[i] = mean(Summarize(chains[i])[1][1:7])
end

end_mchmc = time()
time_mchmc_parallel_ACTPol = end_mchmc - start_mchmc

Let us compute the ESS per second of the parallel MCHMC run.

In [ ]:
ACTPol_MCHMC_parallel_ESS_s = sum(vec_ess)/time_mchmc_parallel_ACTPol

This is higher by a factor of 2.5 than the NUTS ESS/s!

In [ ]:
_chains = zeros(nsteps, n_parallel_mchmc, 7)
for i in 1:n_parallel_mchmc
    _chains[:,i,:] = mapreduce(permutedims, vcat, chains[i])[:,1:7]
end

In [ ]:
actpol_mchmc = mapreduce(permutedims, vcat, actpol_mchmc)
E = actpol_mchmc[:, end-1]
std(E).^2/d

In [ ]:
x = [mapreduce(permutedims, vcat, chains[i]) for i in 1:n_parallel_mchmc]

actpol_mchmc_multi_chains = zeros(nsteps*n_parallel_mchmc, 7)
for i in 1:7
    actpol_mchmc_multi_chains[:,i] = extract_single(x, i, n_parallel_mchmc)
end

In [ ]:
npzwrite("chains_ACT_MCHMC.npy", actpol_mchmc)
npzwrite("chains_ACT_MCHMC_multi.npy", actpol_mchmc_multi_chains)